In [ ]:
!nvidia-smi  # Check GPU information
!nvcc --version  # Check CUDA version

/bin/bash: line 1: nvidia-smi: command not found
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%writefile bfs.cu
#include <stdio.h>
#include <stdlib.h>
#include <vector>
#include <queue>

#define THREADS_PER_BLOCK 256

__global__ void bfs_kernel(int* edges, int* edge_offsets, int* visited, int* levels, int node_count, int current_level) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < node_count && visited[idx] == 1 && levels[idx] == current_level) {
        int start = edge_offsets[idx];
        int end = edge_offsets[idx + 1];

        for (int i = start; i < end; i++) {
            int neighbor = edges[i];
            if (visited[neighbor] == 0) {
                visited[neighbor] = 1;
                levels[neighbor] = current_level + 1;
            }
        }
    }
}

void bfs_cuda(const std::vector<std::vector<int>>& graph, int start_node) {
    int node_count = graph.size();

    // Convert to CSR format
    std::vector<int> edges;
    std::vector<int> edge_offsets(node_count + 1);

    edge_offsets[0] = 0;
    for (int i = 0; i < node_count; i++) {
        edges.insert(edges.end(), graph[i].begin(), graph[i].end());
        edge_offsets[i + 1] = edge_offsets[i] + graph[i].size();
    }

    // Device pointers
    int *d_edges, *d_edge_offsets, *d_visited, *d_levels;

    // Allocate device memory
    cudaMalloc(&d_edges, edges.size() * sizeof(int));
    cudaMalloc(&d_edge_offsets, edge_offsets.size() * sizeof(int));
    cudaMalloc(&d_visited, node_count * sizeof(int));
    cudaMalloc(&d_levels, node_count * sizeof(int));

    // Copy data to device
    cudaMemcpy(d_edges, edges.data(), edges.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_edge_offsets, edge_offsets.data(), edge_offsets.size() * sizeof(int), cudaMemcpyHostToDevice);

    // Initialize visited and levels
    std::vector<int> visited(node_count, 0);
    std::vector<int> levels(node_count, -1);
    visited[start_node] = 1;
    levels[start_node] = 0;

    cudaMemcpy(d_visited, visited.data(), node_count * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_levels, levels.data(), node_count * sizeof(int), cudaMemcpyHostToDevice);

    // Run BFS
    int current_level = 0;
    bool changed;

    do {
        changed = false;

        // Launch kernel
        int blocks = (node_count + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
        bfs_kernel<<<blocks, THREADS_PER_BLOCK>>>(d_edges, d_edge_offsets, d_visited, d_levels, node_count, current_level);
        cudaDeviceSynchronize();

        // Check if we need another iteration
        cudaMemcpy(visited.data(), d_visited, node_count * sizeof(int), cudaMemcpyDeviceToHost);
        for (int i = 0; i < node_count; i++) {
            if (levels[i] == -1 && visited[i] == 1) {
                levels[i] = current_level + 1;
                changed = true;
            }
        }

        if (changed) {
            cudaMemcpy(d_levels, levels.data(), node_count * sizeof(int), cudaMemcpyHostToDevice);
            current_level++;
        }
    } while (changed);

    // Copy results back
    cudaMemcpy(levels.data(), d_levels, node_count * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    printf("BFS Results:\n");
    for (int i = 0; i < node_count; i++) {
        printf("Node %d: Level %d\n", i, levels[i]);
    }

    // Cleanup
    cudaFree(d_edges);
    cudaFree(d_edge_offsets);
    cudaFree(d_visited);
    cudaFree(d_levels);
}

int main() {
    // Example graph (adjacency list)
    std::vector<std::vector<int>> graph = {
        {1, 2},    // Node 0
        {0, 3},     // Node 1
        {0, 3},     // Node 2
        {1, 2, 4},  // Node 3
        {3}         // Node 4
    };

    // Run BFS from node 0
    bfs_cuda(graph, 0);

    return 0;
}

Writing bfs.cu


In [ ]:
!nvcc bfs.cu -o bfs
!./bfs

BFS Results:
Node 0: Level 0
Node 1: Level -1
Node 2: Level -1
Node 3: Level -1
Node 4: Level -1
